<a href="https://colab.research.google.com/github/millenagena/Challenge-Data-Science-Alura-2ed/blob/main/1-Transformando-os-dados/transformacao_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 1
### Contextualização

Como pessoa engenheira de dados temos como uma de nossas responsabilidades a disponibilização dos dados para que os usuários interessados possam realizar suas análises.
<br>
<br>
Na imobiliária InsightPlaces temos um website que torna público os anúncios dos imóveis que pertencem ao portfólio da imobiliária. Este site é alimentado por uma API REST que consulta nosso sistema de banco de dados e retorna um arquivo JSON que é tratado pelo frontend e exibido na tela do usuário. A imobiliária utiliza como SGBD deste recurso (anúncios) o MongoDB que é um SGBD noSQL, ou seja, ele trabalha com dados não estruturados e suas collections são armazenadas no formato JSON.
<br>
<br>
Nosso cliente nessa demanda é a equipe de ciência de dados da InsightPlaces que precisa de um conjunto de dados para criar um modelo de machine learning para possibilitar a criação de um projeto da empresa que consiste em uma ferramenta de avaliação de imóveis usados.

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b0e3612932ef809bf6dfb2e8102c24026f98a52e650780137d440a069bfaf80c
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Transformacao com Spark") \
    .getOrCreate()

spark

## Fazendo o download da base de dados

In [3]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip' && unzip semana-1.zip -d dados/

--2022-11-28 19:53:51--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.77.180, 52.216.244.132, 52.217.64.84, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.77.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  41.6MB/s    in 0.4s    

2022-11-28 19:53:52 (41.6 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


## Realizando a leitura da base de dados

In [4]:
dados = spark.read.json(
    'dados/dataset_bruto.json'
)

In [5]:
dados

DataFrame[anuncio: struct<andar:bigint,area_total:array<string>,area_util:array<string>,banheiros:array<bigint>,caracteristicas:array<string>,endereco:struct<bairro:string,cep:string,cidade:string,estado:string,latitude:double,longitude:double,pais:string,rua:string,zona:string>,id:string,quartos:array<bigint>,suites:array<bigint>,tipo_anuncio:string,tipo_unidade:string,tipo_uso:string,vaga:array<bigint>,valores:array<struct<condominio:string,iptu:string,tipo:string,valor:string>>>, imagens: array<struct<id:string,url:string>>, usuario: struct<id:string,nome:string>]

In [6]:
dados.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

In [7]:
dados.count()

89083

## Avalie a estrutura do arquivo JSON

In [8]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

## Crie um novo DataFrame com todos os campos da coluna `anuncio`.
Cada campo da coluna `anuncio` deve ser uma coluna do novo DataFrame.

Observe que temos uma série de problemas para resolver. Colunas cujo conteúdo são listas, outra com dicionários e uma com dicionários como elementos de uma lista. Vamos tratar cada caso separadamente com o auxílio do dicionário de dados.

### Dicionário de Dados

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |


In [9]:
dados\
    .select(
        'anuncio.*'
    )\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [10]:
anuncio=dados.select('anuncio.*')

## Avalie a coluna `tipo_uso`

Crie uma tabela de frequências segundo o tipo de uso dos imóveis

In [11]:
anuncio\
    .select('tipo_uso')\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



Como a formação de preços de imóveis residenciais e comerciais seguem padrões diferentes, a equipe de cientistas de dados solicitou que fossem apenas coletadas informações para imóveis residenciais. Remova os registros do tipo comercial de nosso DataFrame.

In [12]:
anuncio=anuncio.filter('tipo_uso=="Residencial"')

In [13]:
anuncio\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
+-----------+-----+



## Avalie a coluna `tipo_unidade`

Crie uma tabela de frequências segundo o tipo de imóveis

In [14]:
anuncio\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



Temos três categorias (apartamento, casa e outros). A equipe de cientistas de dados solicitou apenas a extração de informações sobre APARTAMENTOS. Vamos fazer a filtragem apenas para estes tipo e atualizar nosso DataFrame.

In [15]:
anuncio=anuncio.filter('tipo_unidade=="Apartamento"')

In [16]:
anuncio\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66797|
+------------+-----+



## Avaliando as informações de configuração dos imóveis

As informações de configuração dos imóveis podem ser obtidas nas colunas `quartos`, `suites`, `banheiros`, `vaga`, `area_total`, `area_util`. Note que elas vêm em formato de listas e isso tem um motivo.

A imobiliário InsightPlaces trabalha com dois tipos de imóveis. Imóveis usados e imóveis novos (lançamentos). Os imóveis novos geralmente são anunciados ainda em fase de construção e quando são feitos anúncios deste tipo de imóvel o que realmente é anunciado é o empreendimento e não apenas uma das unidades do empreendimento. Acontece que em um empreendimento pode ocorrer de existir mais de um tipo de configuração para os imóveis (exemplo: imóveis de 2 a 4 quartos).

Para dar suporte a este tipo de anúncio os campos acima citados são configurados como listas e no caso de imóveis usados são preenchidos apenas com um valor (exemplo: imóvel de 2 quartos -> [2]) e no caso dos lançamentos podem ser preenchidos com apenas um valor (quando o empreendimento tem imóveis com a mesma configuração) ou com dois valores (quando o empreendimento apresenta imóveis com mais configurações. Exemplos: imóveis de 2 a 4 quartos -> [2, 4]).

Vamos fazer um filtro segundo o tipo de anúncio para mostrar apenas os imóveis do tipo LANÇAMENTO e verificar a ocorrência destes casos de empreendimentos com imóveis com mais de uma configuração (vamos selecionar apenas as colunas `quartos`, `suites`, `banheiros`, `vaga`, `area_total`, `area_util`).

In [17]:
anuncio\
    .select('quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util')\
    .filter('tipo_anuncio=="Lançamento"')\
    .show()

+-------+------+---------+------+----------+---------+
|quartos|suites|banheiros|  vaga|area_total|area_util|
+-------+------+---------+------+----------+---------+
| [1, 2]|   [0]|      [0]|   [0]|        []| [41, 51]|
| [1, 2]|   [0]|      [0]|   [0]|        []|     [41]|
|    [0]|   [0]|      [1]|   [0]|        []| [21, 22]|
|    [0]|   [0]|      [0]|   [0]|        []|[20, 159]|
|    [0]|   [0]|      [0]|   [0]|        []|[22, 150]|
|    [0]|   [0]|      [0]|   [1]|        []| [20, 51]|
|    [0]|   [0]|      [0]|   [0]|        []|[27, 202]|
| [1, 2]|   [1]|      [2]|   [1]|        []|[39, 112]|
|    [2]|   [0]|      [1]|   [1]|        []| [41, 45]|
| [2, 3]|   [0]|      [1]|   [1]|        []| [43, 52]|
|    [2]|   [0]|      [1]|   [1]|        []| [41, 45]|
| [1, 2]|[1, 2]|   [1, 2]|[1, 2]|        []| [30, 66]|
|    [2]|   [0]|      [1]|   [1]|        []| [43, 48]|
| [1, 2]|   [0]|      [1]|   [1]|        []| [35, 40]|
| [1, 2]|   [0]|      [1]|[0, 1]|        []| [32, 43]|
| [1, 2]|[

Neste contexto precisamos acrescentar duas etapas em nosso pipeline de extração e tratamento dos dados para a equipe de ciência de dados da imobiliária. Primeiro precisamos eliminar os registros de lançamentos e depois transformar os dados em formato de lista para o formato numérico.

O primeiro procedimento foi um pedido do próprio time de cientistas de dados, visto que o objetivo do projeto e avaliar imóveis usados e o segundo procedimento seria um tratamento básico para possibilitar a utilização dos dados por algoritmos de machine learning.

## Selecione apenas os imóveis usados

In [18]:
anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



In [19]:
anuncio=anuncio.filter('tipo_anuncio=="Usado"')

In [20]:
anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



## Transforme as colunas de configuração dos imóveis de listas para inteiros

In [21]:
from pyspark.sql import functions as f

### Passo 1 - Ter certeza que temos apenas listas com apenas um elemento

In [22]:
anuncio\
    .select(f.size(f.col('quartos')).alias('quartos'))\
    .groupBy('quartos')\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+



In [23]:
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [24]:
for col in caracteristicas:
    anuncio.select(f.size(f.col(col)).alias(col)).groupBy(col).count().show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



Verificamos que as colunas `suites`, `vaga` e `area_total` apresentam listas vazias em alguns registros. No caso das colunas `suites` e `vagas` podemos considerar que a falta de informação indica que este atributo não existe no imóvel e portanto podemos inputar o valor zero. No caso da coluna `area_total` pode ser resultado de outros fatores.

### Passo 2 - Substituir as listas pelos valores

In [25]:
anuncio\
    .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [26]:
anuncio=anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

In [27]:
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

## Tratamento para as informações sobre localização

A InsightPlaces opera apenas com imóveis na cidade do Rio de Janeiro e a equipe de ciência de dados nos solicitou que apenas as informações sobre bairro e zona da cidade fossem extraídas. Vamos analisar a coluna `endereco` e transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.

In [28]:
anuncio\
    .select('endereco.*')\
    .show()

+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|
+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|R

In [29]:
anuncio\
    .select('endereco.bairro', 'endereco.zona')\
    .show()

+--------------------+----------+
|              bairro|      zona|
+--------------------+----------+
|           Paciência|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|  Pedra de Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|        Campo Grande|Zona Oeste|
|          Santa Cruz|Zona Oeste|
|           Guaratiba|Zona Oeste|
|Recreio dos Bande...|Zona Oeste|
|              Cosmos|Zona Oeste|
|            Realengo|Zona Oeste|
|              Cosmos|Zona Oeste|
|     Todos os Santos|Zona Norte|
+--------------------+----------+
only showing top 20 rows



In [30]:
anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|[P

In [31]:
anuncio=anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')

## Tratamento para a coluna de valores

Segundo o time de cientistas de dados essa é a informação mais importante. Na verdade temos mais de uma informação neste campo. Vamos entender sua estrutura.

```
root
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
```

Note que valores é do tipo array, ou seja, uma lista que contém elementos. E estes elementos são dicionários com as informações de `condominio`, `iptu`, `tipo`, e `valor`

A InsightPlaces permite que o anunciante crie um anúncio com duas opções de valor. O cliente pode criar um anúncio que mostre tanto o valor de venda do imóvel como também o seu valor de locação, juntamente com os valores de taxa de condomínio (quando houver) e taxa de IPTU. Estes valores são diferenciado pelo campo `tipo` que pode assumir os valores `Venda` e `Aluguel`.

Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo VENDA.

Selecione apenas os valores de venda com os respectivos valores de condomínio e iptu.

In [32]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



In [33]:
anuncio\
    .select('valores')\
    .show()

+--------------------+
|             valores|
+--------------------+
|[{245, null, Vend...|
|[{0, 0, Venda, 15...|
|[{0, 0, Venda, 20...|
|[{285, null, Vend...|
|[{245, null, Vend...|
|[{285, null, Vend...|
|[{250, null, Vend...|
|[{245, null, Vend...|
|[{245, null, Vend...|
|[{240, null, Vend...|
|[{0, 0, Venda, 15...|
|[{240, 0, Venda, ...|
|[{245, null, Vend...|
|[{290, null, Vend...|
|[{285, null, Vend...|
|[{0, 0, Venda, 30...|
|[{null, null, Ven...|
|[{280, 0, Venda, ...|
|[{280, null, Vend...|
|[{0, 0, Venda, 26...|
+--------------------+
only showing top 20 rows



In [34]:
anuncio\
    .select(f.explode('valores').alias('valores'))\
    .select('valores.*')\
    .filter('valores.tipo=="Venda"')\
    .count()

66562

In [35]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .show(100, truncate=False)

+------------------------------------+----------+----+-------+-----+
|id                                  |condominio|iptu|tipo   |valor|
+------------------------------------+----------+----+-------+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda  |15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda  |15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda  |20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda  |20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda  |15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda  |20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda  |15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda  |15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda  |15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda  |17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda  |15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12

In [36]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')\
    .show(truncate=False)

+------------------------------------+----------+----+-----+-----+
|id                                  |condominio|iptu|tipo |valor|
+------------------------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda|15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda|15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda|20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda|20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda|15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda|20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda|15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda|15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda|15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda|17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda|15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12014|240       |0   |Venda|19

In [37]:
valores=anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')

### Juntando os valores com as outras informações do DataFrame

In [38]:
anuncio.join(valores, 'id', how='inner').count()

66562

In [39]:
anuncio.join(valores, 'id', how='inner').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|[{285, null, Vend...|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado|

In [40]:
anuncio.join(valores, 'id', how='inner').drop('valores').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [41]:
anuncio=anuncio.join(valores, 'id', how='inner').drop('valores')

## Salvando o resultado da extração no formato parquet

In [42]:
anuncio.write.parquet(
    path='dados_salvos/dataset_transformado_parquet',
    mode='overwrite'
)

In [43]:
anuncio_parquet = spark.read.parquet(
    'dados_salvos/dataset_transformado_parquet'
)

In [44]:
anuncio_parquet.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [45]:
anuncio_parquet.count()

66562

## Desafio extra: Salvar os dados no formato csv

In [46]:
anuncio = anuncio.drop("caracteristicas")

In [47]:
anuncio.write.csv(
    path='dados_salvos/dataset_transformado.csv',
    mode='overwrite',
    header=True
)

In [48]:
anuncio_csv = spark.read.csv(
    'dados_salvos/dataset_transformado.csv',
    header=True
)

In [49]:
anuncio_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: string (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: string (nullable = true)
 |-- quartos: string (nullable = true)
 |-- suites: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## Desafio extra: Comparar o desempenho de arquivos parquet e csv

In [50]:
%%time
anuncio_parquet.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|       Tomás Coelho|   38|
|              Gávea|  597|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|      Bento Ribeiro|   33|
|              Rocha|   38|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|       Campo Grande|  639|
|        Jacarepaguá| 4437|
|               Leme|  234|
|           Botafogo| 3499|
|       Padre Miguel|   15|
|       Santo Cristo|  357|
|           Flamengo| 1378|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 6.08 ms, sys: 850 µs, total: 6.93 ms
Wall time: 343 ms


In [51]:
%%time
anuncio_csv.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|       Tomás Coelho|   38|
|              Gávea|  597|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|      Bento Ribeiro|   33|
|              Rocha|   38|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|       Campo Grande|  639|
|        Jacarepaguá| 4437|
|               Leme|  234|
|           Botafogo| 3499|
|       Padre Miguel|   15|
|       Santo Cristo|  357|
|           Flamengo| 1378|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 12.7 ms, sys: 384 µs, total: 13.1 ms
Wall time: 911 ms
